### Customer Churn Prediction for SyriaTel: A Binary Classification Approach

## 1. Business Understanding 

### 1.1 Problem Statement

SyriaTel is a mobile telecommunications company that has built an unquestionable reputation in the industry over the years. It has established a strong and consistent source of revenue, thanks to its unrivaled market dominance.

Like any other telecommunication business operating in a competitive environment, the telecommunication giant is taking precautionary measures to cushion itself against potential risks - particularly if the competition intensifies and some customers decide to switch loyalty to other emerging service providers. In addition, the company is interested in the factors that make customers switch loyalty, come up with customer retention policies, and maximize profits.

Therefore, as a senior data scientist of the telecommunication giant, I have been tasked to come up with a predictive machine learning model to evaluate whether a customer will churn SyriaTel or remain loyal to the company in the near future.

### 1.2 Business Objectives
* Come up with a model to predict whether a customer churns or stays.
* Analyze the factors contributing to churning.

### 1.3 Success Metrics
